# 1. Import the necessary libraries for you to be able to query 4square API

In [1]:
import requests
import json
import pandas as pd
from getpass import getpass

In [2]:
import folium
from folium import Choropleth, Circle, Marker, Icon, Map
from folium.plugins import HeatMap, MarkerCluster
import pandas as pd

In [3]:
import os
import requests
import json
from dotenv import load_dotenv
import pandas as pd
from getpass import getpass
import geopandas as gpd
from cartoframes.viz import Map, Layer, popup_element
from dotenv import load_dotenv

In [4]:
load_dotenv()

True

# 2. Get names and location of the 10 coffee places by passing Ironhack as your location

In [5]:
# your code here
token = getpass()

In [6]:
where = "Carrer de pamplona, 96, 08018, Barcelona, Spain"

In [7]:
url_geocode = f"https://geocode.xyz/{where}?json=1"
url_geocode

'https://geocode.xyz/Carrer de pamplona, 96, 08018, Barcelona, Spain?json=1'

In [8]:
requests.get(url_geocode).json()

{'standard': {'stnumber': '96',
  'addresst': 'Carrer de Pamplona',
  'statename': 'Catalonia',
  'postal': '08018',
  'region': 'Catalonia',
  'prov': 'ES',
  'city': 'Barcelona',
  'countryname': 'Spain',
  'confidence': '0.90'},
 'longt': '2.18997',
 'alt': {},
 'elevation': {},
 'latt': '41.39789'}

In [9]:
bar_long = 2.18997
bar_lat = 41.39789

In [10]:
query = "Cafeteria"

In [44]:
import requests

url = "https://api.foursquare.com/v3/places/search?query=coffee&limit=10"

headers = {
    "accept": "application/json",
    "Authorization": token
}

response = requests.get(url, headers=headers)

In [47]:
response.json()

{'results': [{'fsq_id': '58a196a45cab2f7fd67c7384',
   'categories': [{'id': 13034,
     'name': 'Café',
     'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/cafe_',
      'suffix': '.png'}},
    {'id': 13035,
     'name': 'Coffee Shop',
     'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/coffeeshop_',
      'suffix': '.png'}}],
   'chains': [],
   'distance': 311,
   'geocodes': {'main': {'latitude': 41.384525, 'longitude': 2.168502},
    'roof': {'latitude': 41.384525, 'longitude': 2.168502}},
   'link': '/v3/places/58a196a45cab2f7fd67c7384',
   'location': {'address': 'Calle de Les Ramelleres, 16',
    'admin_region': 'Cataluña',
    'country': 'ES',
    'cross_street': '',
    'formatted_address': 'Calle de Les Ramelleres, 16, 08001 Barcelona Catalonia',
    'locality': 'Barcelona',
    'postcode': '08001',
    'region': 'Catalonia'},
   'name': 'Dalston Coffee',
   'related_places': {},
   'timezone': 'Europe/Madrid'},
  {'fsq_id': '5cd80cf92bf29b0

In [48]:
coordinates = response.json()["results"][0]["geocodes"]["main"]
coordinates

{'latitude': 41.384525, 'longitude': 2.168502}

In [49]:
lat, lon = coordinates["latitude"], coordinates["longitude"]

In [50]:
name = response.json()["results"][0]["name"]
name

'Dalston Coffee'

In [51]:
def name_coordinates (dict_):
    
    processed_dict = {"name": dict_["name"],
                     "lat": dict_["geocodes"]["main"]["latitude"],
                     "lon": dict_["geocodes"]["main"]["longitude"]}
    
    return processed_dict

In [52]:
name_coordinates (response.json()["results"][0])

{'name': 'Dalston Coffee', 'lat': 41.384525, 'lon': 2.168502}

In [53]:
new_list = []
for i in response.json()["results"]:
    new_list.append(name_coordinates(i))

In [54]:
df = pd.DataFrame(new_list)
df

,name,lat,lon
0,Dalston Coffee,41.384525,2.168502
1,Barcentral,41.383680,2.168826
2,Chök,41.384669,2.168349
3,Chok - The Chocolate Kitchen,41.382801,2.171393
4,Departure Coffee Co,41.383610,2.164875
5,Nømad Coffee Lab & Shop,41.388989,2.177167
6,Caravelle,41.382226,2.168842
7,La Masala Cafe,41.388319,2.177882
8,Churrería Laietana,41.386450,2.175654
9,Salterio,41.382539,2.176081


3. Now get the 5 CLOSEST coffe places. Make use of params: limit, near?

In [90]:
url = "https://api.foursquare.com/v3/places/search?query=coffee&ll=41.384525%2C2.168502&radius=1000&limit=5"

headers = {
    "accept": "application/json",
    "Authorization": token
}

response = requests.get(url, headers=headers)


In [91]:

response.json()

{'results': [{'fsq_id': '58a196a45cab2f7fd67c7384',
   'categories': [{'id': 13034,
     'name': 'Café',
     'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/cafe_',
      'suffix': '.png'}},
    {'id': 13035,
     'name': 'Coffee Shop',
     'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/coffeeshop_',
      'suffix': '.png'}}],
   'chains': [],
   'distance': 14,
   'geocodes': {'main': {'latitude': 41.384525, 'longitude': 2.168502},
    'roof': {'latitude': 41.384525, 'longitude': 2.168502}},
   'link': '/v3/places/58a196a45cab2f7fd67c7384',
   'location': {'address': 'Calle de Les Ramelleres, 16',
    'admin_region': 'Cataluña',
    'country': 'ES',
    'cross_street': '',
    'formatted_address': 'Calle de Les Ramelleres, 16, 08001 Barcelona Catalonia',
    'locality': 'Barcelona',
    'postcode': '08001',
    'region': 'Catalonia'},
   'name': 'Dalston Coffee',
   'related_places': {},
   'timezone': 'Europe/Madrid'},
  {'fsq_id': '56d728de498e1991

In [92]:
coordinates = response.json()["results"][0]["geocodes"]["main"]


In [93]:
lat, lon = coordinates["latitude"], coordinates["longitude"]

In [94]:
name = response.json()["results"][0]["name"]
name

'Dalston Coffee'

In [95]:
def name_coordinates (dict_):
    
    processed_dict = {"name": dict_["name"],
                     "lat": dict_["geocodes"]["main"]["latitude"],
                     "lon": dict_["geocodes"]["main"]["longitude"]}
    
    return processed_dict

In [96]:
name_coordinates (response.json()["results"][0])

{'name': 'Dalston Coffee', 'lat': 41.384525, 'lon': 2.168502}

In [97]:
new_list = []
for i in response.json()["results"]:
    new_list.append(name_coordinates(i))

In [98]:
df = pd.DataFrame(new_list)
df

,name,lat,lon
0,Dalston Coffee,41.384525,2.168502
1,Chök,41.384669,2.168349
2,Barcentral,41.383680,2.168826
3,Buenas Migas,41.383811,2.169646
4,Bubbolitas,41.384116,2.169800


# 4. Define a function that returns name, lat, long for starbucks OR restaurants AND for any given location

In [112]:

def foursquare_places(location, venue):
    
    url = "https://api.foursquare.com/v3/places/search?query=coffee&chains=	ab4c54c0-d68a-012e-5619-003048cad9da&limit=10" #your_url
    
    headers = {
        "accept": "application/json",
        "Authorization": token
    }
    
    response = requests.get(url, headers=headers)

    if response.status_code != 200:
        return None

    def name_coordinates(dict_):
        processed_dict = {"name": dict_["name"],
                          "lat": dict_["location"]["lat"],
                          "lon": dict_["location"]["lng"]}
        return processed_dict

    new_list = []
    for i in response.json()["results"]:
        new_list.append(name_coordinates(i))

    return new_list


In [115]:
def name_coordinates (dict_):  
    processed_dict = {"name": dict_["name"],
                     "lat": dict_["geocodes"]["main"]["latitude"],
                     "lon": dict_["geocodes"]["main"]["longitude"]}
    return processed_dict

In [132]:
def foursquare_places (venue, coordinates):
    url = f"https://api.foursquare.com/v3/places/search?query={venue}&ll={coordinates}1&limit=10"
    headers = {
    "accept": "application/json",
    "Authorization": (token)
    }
    response = requests.get(url, headers=headers)
       
    coordinates = response.json()["results"][0]["geocodes"]["main"]
    lat, lon = coordinates["latitude"], coordinates["longitude"]
    name = response.json()["results"][0]["name"]
    name_coordinates(response.json()["results"][0])
    
    new_list2 = []
    for i in response.json()["results"]:
        new_list2.append(name_coordinates(i))
        
    return pd.DataFrame(new_list2)




In [133]:
restaurants = foursquare_places ("restaurantes", "40.4169335,-3.7083759")

restaurants

,name,lat,lon
0,Restaurante SoMa,40.425047,-3.678081
1,Restaurante Soria,40.402668,-3.690501
2,"Restaurante Si, No",40.420613,-3.698860
3,PadThaiWok. Malasaña,40.423051,-3.703115
4,Padthaiwok,40.411486,-3.709060
5,Restaurante Sidrería Carlos Tartiere,40.419490,-3.675066
6,Restaurante Samarkanda,40.407153,-3.691402
7,Restaurante Sombrero Azul,40.422968,-3.708127
8,Restaurante Salmantino,40.428669,-3.708941
9,Restaurantes.com,40.434511,-3.711609


# 5. Export it into data folder as a json: `data/coffe_shops.json`

In [134]:
restaurants.to_json("/Users/victorramos/documents/ironhack/labs/week 4/lab-coffee-shops/restaurants.json")

# BONUS: load it into MongoDB in a collection named "ironcoffee" using pymongo

In [19]:
# your code here
